In [15]:
!pip install networkx[default]

Defaulting to user installation because normal site-packages is not writeable


In [16]:
!git clone -b data https://github.com/tu-leminh/Community-detection.git

fatal: destination path 'Community-detection' already exists and is not an empty directory.


In [17]:
import networkx as nx
from networkx.algorithms import community
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession .builder .appName("Task 2") .getOrCreate()

In [18]:
df = spark.read.csv("Community-detection/ub_sample_data.csv",header=True)

In [19]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[user_id: string, business_id: string]>

In [20]:
df.show(3,truncate=False)

+----------------------+----------------------+
|user_id               |business_id           |
+----------------------+----------------------+
|39FT2Ui8KUXwmUt6hnwy-g|RJSFI7mxGnkIIKiJCufLkg|
|39FT2Ui8KUXwmUt6hnwy-g|fThrN4tfupIGetkrz18JOg|
|39FT2Ui8KUXwmUt6hnwy-g|mvLdgkwBzqllHWHwS-ZZgQ|
+----------------------+----------------------+
only showing top 3 rows



In [21]:
df=df.select(F.col("user_id").alias("x"),F.col('business_id')).join(df.select(F.col("user_id").alias("y"),F.col('business_id')),on="business_id",)
df.show(3,truncate=False)

+----------------------+----------------------+----------------------+
|business_id           |x                     |y                     |
+----------------------+----------------------+----------------------+
|RJSFI7mxGnkIIKiJCufLkg|39FT2Ui8KUXwmUt6hnwy-g|Et6mwzOORak5K6kJ-yw4Xw|
|RJSFI7mxGnkIIKiJCufLkg|39FT2Ui8KUXwmUt6hnwy-g|jfaz1aQqnkbtzQfQyCE8IA|
|RJSFI7mxGnkIIKiJCufLkg|39FT2Ui8KUXwmUt6hnwy-g|a0PH_2zKVQExMP5w-OJQKw|
+----------------------+----------------------+----------------------+
only showing top 3 rows



In [22]:
df=df.groupBy(["x","y"]).count()
df.show(3)

+--------------------+--------------------+-----+
|                   x|                   y|count|
+--------------------+--------------------+-----+
|39FT2Ui8KUXwmUt6h...|WPcEBAbrPZxvCGlvH...|    1|
|39FT2Ui8KUXwmUt6h...|7G8w2SnaC-qDVQ7_G...|    1|
|39FT2Ui8KUXwmUt6h...|fYJf6G30hZlhVxWTO...|    2|
+--------------------+--------------------+-----+
only showing top 3 rows



In [23]:
df=df.filter("count >= 7").select("x","y").filter("x!=y")
df.show(3)

+--------------------+--------------------+
|                   x|                   y|
+--------------------+--------------------+
|TZ974xcbw2kqjYxAh...|JM0GL6Dx4EuZ1mprL...|
|fLnkI1uHtXEsjtF6K...|MwpK7PqQX7fgTFM2P...|
|7Vfy39A_totC-w70q...|0FVcoJko1kfZCrJRf...|
+--------------------+--------------------+
only showing top 3 rows



In [24]:
G = nx.Graph()
for row in df.collect():
    G.add_node(row["x"])
    G.add_node(row["y"])
    G.add_edge(row["x"], row["y"])
G=G.to_undirected()

In [25]:
def rm_until_new_com(G):
    pre=nx.number_connected_components(G)
    while(nx.number_connected_components(G)==pre):
        dic = nx.edge_betweenness_centrality(G)
        edge=max(dic, key=dic.get)
        G.remove_edge(*edge)
    return(G)


def gn_algorithm(G,k=1):
    if len(G.edges)==0:
        return G
    g=G.copy()
    for i in range(k):
        if (len(g.edges)<=0): continue
        rm_until_new_com(g)
    return list(nx.connected_components(g))

In [26]:
tempt=sorted([len(i) for i in gn_algorithm(G,1)],reverse=True)

In [27]:
#https://networkx.guide/algorithms/community-detection/girvan-newman/
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman

communities = girvan_newman(G)

node_groups = []
for com in next(communities):
    node_groups.append(len(com))

print(sorted(node_groups,reverse=True),tempt==sorted(node_groups,reverse=True))

[132, 58, 5, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2] True


In [31]:
print(sorted([len(i) for i in gn_algorithm(G,15)],reverse=True))

[38, 33, 25, 16, 14, 14, 11, 9, 6, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1]
